In [32]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm_notebook
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression, mutual_info_classif, chi2, f_classif


In [34]:
choose_element = 'Fe2O3' #['SiO2','Fe2O3','Al2O3','CaO','MgO','K2O','Na2O','TiO2']
#for coeffs comparison
save_path = r'D:\data\china_2020\coeffs_diff'
data_file_path = r'D:\data\china_2020\coeffs_mean'
menu_guide_path = r'D:\data\Menu_Guide-quantitative.xlsx'
process_path = ['DenoisedSpectrum', 'SmoothedSpectrum', 'RawSpectrum']
file_list = 'Spectra_Name'
data_flag = pd.read_excel(menu_guide_path)
file_names = np.array(data_flag[file_list])
concentration = np.array(data_flag[choose_element])
########################################################################################################################

train_position = [1, 3, 12, 13, 21]

for process in tqdm_notebook(process_path):
    for index_i in tqdm_notebook(train_position):
        temp_concentration = concentration[index_i]
        data_diff = np.zeros((8, 127, 4559))
        for j in range(8):
            data_cache = {'rock': 0, 'pellet': 0}
            for state in ['rock', 'pellet']:
                data_cache[state] = np.loadtxt(os.path.join(r'D:\data\china_2020\coeffs_mean', state+'_'+file_names[index_i]+'_'+process+'_'+str(j)+'.txt'))
            data_diff[j] = data_cache['rock'] - data_cache['pellet']
        data_diff_std = np.std(data_diff, axis=0)
        data_diff_mean = np.sum(data_diff, axis=0) / 8
        np.savetxt(os.path.join(save_path, file_names[index_i]+'_'+process+'.txt'), data_diff_mean)
        np.savetxt(os.path.join(save_path, file_names[index_i]+'_'+process+'_std.txt'), data_diff_std)

In [ ]:
for choose_element in tqdm_notebook(['SiO2','Fe2O3','Al2O3','CaO','MgO','K2O','Na2O','TiO2']):
    #for coeffs comparison
    save_path = r'D:\data\china_2020\coeffs_diff_scores'
    data_file_path = r'D:\data\china_2020\coeffs_diff'
    menu_guide_path = r'D:\data\Menu_Guide-quantitative.xlsx'
    process_path = ['DenoisedSpectrum', 'SmoothedSpectrum', 'RawSpectrum']
    file_list = 'Spectra_Name'
    data_flag = pd.read_excel(menu_guide_path)
    file_names = np.array(data_flag[file_list])
    concentration = np.array(data_flag[choose_element])
    ########################################################################################################################

    train_position = [1, 3, 12, 13, 21]

    process = 'RawSpectrum'
    X_tr = None
    for index_i in tqdm_notebook(train_position):
        temp_concentration = concentration[index_i]
        for j in range(8):
            data = np.loadtxt(os.path.join(data_file_path, file_names[index_i]+'_'+process+'_'+str(j)+'.txt')).flatten()
            if X_tr is None:
                X_tr = data
                y_tr = temp_concentration
            else:
                X_tr = np.row_stack((X_tr, data))
                y_tr = np.row_stack((y_tr, temp_concentration))
    skb = SelectKBest(f_regression, k=X_tr.shape[1])
    X_tr = skb.fit_transform(X_tr, y_tr.ravel())
    score = skb.scores_
    score = score.tolist()
    np.savetxt(os.path.join(save_path, choose_element + '_' + process + '.txt'), score)

In [29]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm_notebook
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression, mutual_info_classif, chi2, f_classif
from sklearn.cluster import KMeans
import sklearn
data_file_path = r'D:\data\china_2020\coeffs'
flag = {'rock': 1, 'pellet': 0}
process = 'RawSpectrum'
X_tr = None
for state in tqdm_notebook(['rock', 'pellet']):
    for index_i in tqdm_notebook(train_position):
        temp_concentration = flag[state] #rock:1 pellet:0
        for i in range(3):
            for j in range(8):
                data = np.loadtxt(os.path.join(data_file_path, state+'_'+file_names[index_i]+'_'+process+'_'+str(i+1)+'_'+str(j)+'.txt')).flatten()
                if X_tr is None:
                    X_tr = data
                    y_tr = temp_concentration
                else:
                    X_tr = np.row_stack((X_tr, data))
                    y_tr = np.row_stack((y_tr, temp_concentration))


In [31]:
chi2, pval = chi2(X_tr, y_tr)
np.savetxt(os.path.join(r'D:\data\china_2020\coeffs_rock_pellet_chi2', process + '.txt'), np.column_stack((chi2, pval)))

In [28]:
import sklearn
kmeans_model = KMeans(n_clusters=3, random_state=1).fit(X_tr)
labels = kmeans_model.labels_
print('labels = '+str(labels))
sklearn.metrics.silhouette_score(X_tr, labels, metric='euclidean')

labels = [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


0.5639184671755635

In [45]:
orig_path = r'D:\data\china_2020\pixel_select'
for j in tqdm_notebook([1, 3, 12, 13, 21]):
    for i in tqdm_notebook(range(3)):
        for state in ['pellet', 'rock']:
            data_cache = {'Raw': 0, 'Dark': 0}
            for index_i in ['Raw', 'Dark']:
                data_cache[index_i] = np.loadtxt(os.path.join(orig_path, file_names[j]+'-'+state+'_RawSpectrum_'+index_i+'_'+str(i+1)+'.txt'))
                pixel_list = data_cache[index_i][:, 0]
                data_cache[index_i] = data_cache[index_i][:, 1:]
            data_diff = data_cache['Raw'] - data_cache['Dark'].dot(np.ones((1, 51)))
            np.savetxt(os.path.join(orig_path, file_names[j]+'-'+state+'_RawSpectrum_'+str(i+1)+'.txt'), np.column_stack((pixel_list, data_diff)), '%.5g')